# Atelier sur la génétique statistique et l'épidémiologie génétique STAGE-Québec
## Thème 2 - Phénotypes moléculaires en épidémiologie génétique

Par Marc-André Legault (Université de Montréal) et Qihuang Zhang (Université McGill)

**31 juillet - 1er août 2025**

### Introduction

Dans cet atelier, nous effectuerons une analyse d'association à l'échelle du transcriptome (TWAS) pour identifier les gènes du diabète de type 2 (DT2). Plus précisément, nous testerons l'association entre l'expression génique prédite dans divers tissus et le DT2. Nous utiliserons de vraies statistiques d'association sommaires d'une méta-analyse récente impliquant 180 834 cas avec DT2 et plus d'un million de témoins [Mahajan _et al._ (2022)](https://www.nature.com/articles/s41588-022-01058-3). Nous utiliserons à la fois [S-PrediXcan](https://www.nature.com/articles/s41467-018-03621-1) et [TWAS FUSION](https://www.nature.com/articles/ng.3506) pour effectuer les analyses TWAS. Les statistiques sommaires GWAS ont été pré-traitées pour s'assurer qu'elles sont sur la même version du génome (GRCh38) que les modèles de prédiction d'expression génique distribués par ces logiciels. Nous nous concentrerons également sur un sous-ensemble de tissus de GTEx pour réduire la charge computationnelle.

Nous divisons le contenu de l'atelier en trois sections.

1. S-PrediXcan (ce carnet)
2. TWAS / FUSION [(lien)](2a-FUSION.ipynb)
3. Comparaison des deux approches [(lien)](3a-compare_TWAS_models.ipynb)

Il y a un navigateur de fichiers dans le panneau de gauche qui peut être utilisé pour parcourir le contenu de l'atelier.

<div class="alert alert-warning">
    <p>
    Le dossier <strong>local</strong> est partagé avec votre ordinateur. Tout le reste ne sera pas sauvegardé si vous fermez le serveur de carnets ! Assurez-vous que tout ce qui devrait persister est stocké dans le dossier local.
    </p>
    <p>
    Le contenu de ce dossier est stocké dans votre répertoire personnel (par exemple C:\Users\[votre nom d'utilisateur]\STAGE2025_workshop_theme2 ou <br /> /Users/[votre nom d'utilisateur]/STAGE2025_workshop_theme2).
    </p>
</div>

Nous pouvons maintenant commencer par mieux comprendre S-PrediXcan et l'exécuter en utilisant nos données GWAS pré-traitées.

### Exploration des statistiques sommaires GWAS

Les statistiques sommaires GWAS ont été harmonisées avec les modèles [PredictDB](https://predictdb.hakyimlab.org/) qui sont les modèles de prédiction d'expression génique distribués comme données complémentaires à S-PrediXcan. Le format des données est le même pour chaque tissu :

In [ ]:
filename="../data/PrediXcan/gwas_harmonized/harmonized_en_Whole_Blood.tsv.gz"
echo "Looking at file $filename"
echo "Column numbers and titles:"
echo "$(zcat $filename | head -n 1 | tr '\t' '\n' | nl)"
echo
echo "File excerpt:"
zcat $filename | head -n 10 | cut -f 1-6,9

<div class="alert alert-info">
    Interprétez les colonnes affichées.
</div>

### Modèles de prédiction d'expression génique

Maintenant, nous pouvons examiner les modèles de prédiction d'expression génique pour le tissu GTEx « Sang total ». Dans PredictDB, les modèles sont distribués sous forme de bases de données sqlite3. Les bases de données ont deux tables : ``extra`` et ``weights`` avec la structure suivante :

In [ ]:
# Schéma pour la table "extra" :
sqlite3 ../data/PrediXcan/predictdb/en_Whole_Blood.db ".schema extra"

``extra`` contient les métadonnées sur le modèle prédictif pour chaque gène. Vous trouverez le nom du gène, l'ID Ensembl du gène, le nombre de SNP qui ont été inclus dans le modèle ainsi que les statistiques sur la performance prédictive du modèle d'expression génique.

<div class="alert alert-info">
Commentez l'importance de la colonne <code>pred.perf.pval</code>. À quoi sert-elle ?
</div>

La table weights contient l'information sur le modèle d'expression génique. Pour chaque ``gene`` (ID Ensembl du gène), elle listera les variants qui sont utilisés pour prédire l'expression génique (c.-à-d. les eQTL) avec leur ``weight`` de prédiction. Comme exemple concret, nous pouvons extraire le modèle utilisé pour prédire l'expression pour ENSG00000157823.16 (Adaptor Related Protein Complex 3 Subunit Sigma 2, _AP3S2_). Par souci de brièveté, nous trierons les variants par poids absolu, et ne montrerons que les 10 principaux contributeurs à la prédiction.

In [ ]:
sqlite3 ../data/PrediXcan/predictdb/en_Whole_Blood.db <<EOF
.mode columns
.header on
select * from weights
  where gene='ENSG00000157823.16'
  order by abs(weight) desc limit 10;
EOF

<div class="alert alert-info">
Pourquoi certains poids sont-ils positifs et d'autres négatifs ? Pouvez-vous mentionner une façon d'estimer les poids de prédiction ?
</div>

### Analyse TWAS utilisant S-PrediXcan

Maintenant que nous avons exploré la structure des modèles de prédiction d'expression génique PredictDB et que nous avons les statistiques sommaires GWAS, nous pouvons implémenter l'analyse TWAS en utilisant S-PrediXcan. Le code ci-dessous peut prendre quelques minutes à s'exécuter. S'il prend trop de temps, vous pouvez l'arrêter en utilisant le bouton d'arrêt en haut de cette fenêtre, et utiliser les fichiers de points de contrôle disponibles dans le dépôt.

L'appel à SPrediXcan prend 3 fichiers d'entrée :

- model_db_path : Ceci contient les poids pour prédire l'expression des gènes inclus dans le TWAS. C'est la base de données sqlite3 fournie par PredictDB.
- covariance : Ce fichier contient les estimations de DL entre les SNP. Il tient compte de la covariance entre les variants génétiques.
- gwas_file : C'est le fichier de statistiques sommaires GWAS contenant les statistiques d'association entre les variants génétiques et le DT2. Nous avons précédemment harmonisé les résultats GWAS bruts pour nous assurer que les variants étaient cohérents avec ceux disponibles dans PredictDB.

Les paramètres avec le suffixe ``_column`` sont utilisés pour que le logiciel sache comment interpréter le fichier de statistiques sommaires GWAS. Vous pouvez vous convaincre que les noms fournis ci-dessous sont appropriés en regardant à nouveau l'extrait du fichier de statistiques sommaires GWAS ci-dessus.

Finalement, nous fournissons une estimation de l'héritabilité SNP ($h^2$) pour notre phénotype et la taille d'échantillon GWAS (N). Ces paramètres permettent au logiciel de recalibrer les statistiques d'association pour tenir compte de la polygénicité, comme décrit dans [Liang _et al. medRxiv_ 2024](https://www.biorxiv.org/content/10.1101/2023.10.17.562831v2). En bref, les traits polygéniques ont de nombreux variants causaux avec des tailles d'effet faibles dispersés à travers le génome. Plusieurs de ces variants seront en DL avec les eQTL inclus dans les modèles de prédiction d'expression génique, ce qui pourrait mener à des associations fallacieuses. Fournir ces statistiques permet à S-PrediXcan de recalibrer les statistiques d'association TWAS pour tenir compte de cet effet.

In [ ]:
tissues="Adipose_Subcutaneous Artery_Coronary Brain_Cortex Liver Muscle_Skeletal Pancreas Whole_Blood"

for tissue in $tissues; do
    echo
    echo
    echo "Running S-PrediXcan for tissue ${tissue}"
    echo
    SPrediXcan.py \
        --model_db_path ../data/PrediXcan/predictdb/en_${tissue}.db \
        --covariance ../data/PrediXcan/predictdb/en_${tissue}.txt.gz \
        --gwas_file ../data/PrediXcan/gwas_harmonized/harmonized_en_${tissue}.tsv.gz \
        --snp_column rsid_x \
        --effect_allele_column effect_allele \
        --non_effect_allele_column other_allele \
        --output_file /workshop/local/results/twas_en_${tissue}.csv \
        --beta_column beta \
        --se_column standard_error \
        --gwas_h2 0.15 \
        --gwas_N 1000000
done

<div class="alert alert-info">
    Vous pouvez ouvrir un des fichiers de résultats dans le répertoire ``results`` pour voir les résultats TWAS par vous-même.
</div>

Dans le [prochain carnet](1b-S-PrediXcan-interpretation.ipynb), nous interpréterons les résultats TWAS que nous avons générés en utilisant le langage R.